In [1]:
#Mcallen
#San Antonio
#Edinburg
#Austin
#Dallas
#San Marcos
#Corpus Christi
#Weslaco
#Boerne
#Houston

### Imports

In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import requests
import pandas as pd
import urllib.parse

### HTTP Request

In [2]:
#store website in a variable
website = 'https://www.trulia.com/TX/McAllen/'

In [3]:
#set up selenium webdriver
driver = webdriver.Chrome()

#get the webpage using selenium

try:
    # Get the webpage using Selenium
    driver.get(website)
    time.sleep(5)  # Give the page some time to load

    # Scroll down to load more content
    for _ in range(20):
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
        time.sleep(1)  # Adjust the delay based on the page loading speed

    # Get the updated page source after scrolling
    updated_html = driver.page_source

    # Create a BeautifulSoup object using the updated HTML
    soup = BeautifulSoup(updated_html, 'html.parser')

    # Extract the desired results with data-testid
    result_container = soup.find_all(attrs={'data-testid': True})


    # Retrieve anchor tags into a list and establish base URL
    url_part_1 = 'https://www.trulia.com'
    url_part_2 = []

    for item in result_container:
        for link in item.find_all('div', {'data-testid': 'property-card-details'}):
            url_part_2.append(link.find('a').get('href'))

finally:
    driver.quit()  # Close the browser window when done

In [4]:
# Remove spaces and duplicates from list
url_only = [x.strip(' ') for x in url_part_2]
url_only = list(dict.fromkeys(url_only))
len(url_only)

40

In [5]:
# Concatenate URL
url_joined = []
for link_2 in url_only:
    url_joined.append(urllib.parse.urljoin(url_part_1,link_2))

In [6]:
url_joined

['https://www.trulia.com/home/8131-n-55th-ln-mcallen-tx-78504-2053645994',
 'https://www.trulia.com/home/3416-toucan-ave-mcallen-tx-78504-123580455',
 'https://www.trulia.com/home/4021-zanesville-ave-mcallen-tx-78503-2055590193',
 'https://www.trulia.com/home/6308-n-20th-st-mcallen-tx-78504-66982997',
 'https://www.trulia.com/home/905-w-sycamore-ave-mcallen-tx-78501-344791067',
 'https://www.trulia.com/home/14620-aqualina-way-mcallen-tx-78504-2059248584',
 'https://www.trulia.com/home/8308-n-35th-st-mcallen-tx-78504-344758557',
 'https://www.trulia.com/home/2604-notre-dame-ave-mcallen-tx-78504-69713277',
 'https://www.trulia.com/builder-community-plan/La-Floresta-Subdivision-The-Delilah-2057295504',
 'https://www.trulia.com/builder-community-plan/Campo-De-Suenos-Monte-2067335390',
 'https://www.trulia.com/home/3010-s-l-ln-mcallen-tx-78503-99401002',
 'https://www.trulia.com/home/3006-s-l-ln-mcallen-tx-78503-344734273',
 'https://www.trulia.com/home/2616-Brazos-Ct-McAllen-TX-78504-10879

### Get data from links

In [9]:
# address
# bedrooms
# bathrooms
# sqft
# year built
# parking
# price

In [7]:
first_link = url_joined[0]
print(first_link)

https://www.trulia.com/home/8131-n-55th-ln-mcallen-tx-78504-2053645994


### Get request and Soup Object

In [8]:
response = requests.get(first_link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})

In [9]:
soup = BeautifulSoup(response.content, 'html.parser')

In [1]:
# Commenting this step out to avoid copyright
#soup

#### Put all of the attributes together and loop through results from page 1


In [11]:
address = []
bedrooms = []
bathrooms = []
area = []
year_built = []
parking = []
price = []

# Loop through all joined links
for link in url_joined:
    response = requests.get(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'})
    
    # Create soup object
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Address
    try:
        address.append(soup.find('span', {'data-testid':'home-details-summary-headline'}).get_text())
    except:
        address.append('')
    # Bedrooms  
    try:
        bedrooms.append(soup.find('li', {'data-testid':'bed'}).get_text())
    except:
        bedrooms.append('')
        
    # Bathrooms
    try:
        bathrooms.append(soup.find('li', {'data-testid':'bath'}).get_text())
    except:
        bathrooms.append('')
        
    # Square Feet
    try:
        area.append(soup.find('li', {'data-testid':'floor'}).get_text())
    except:
        area.append('')
    
    # Year Built
    try:
        year_built.append(soup.find('div',string='Year Built').findNext('span').get_text())
        year_built = [x.strip('Year Built: ') for x in year_built] 
    except:
        year_built.append('')
    
    # Parking
    try:
        parking.append(soup.find(string='Parking').findNext('div').get_text())
    except:
        parking.append('')
    
    # Price
    try:
        price.append(soup.find('h3', {'data-testid':'on-market-price-details'}).get_text())
    except:
        price.append('') 
    
    # Create a dictionary with results
    output = {'Address': address, 'Bathrooms': bathrooms, 'Area':area, 'Year Built':year_built, 
              'Parking':parking, 'Price':price}
    
output

{'Address': ['8131 N  55th Ln',
  '3416 Toucan Ave',
  '4021 Zanesville Ave',
  '6308 N  20th St',
  '905 W  Sycamore Ave',
  '14620 Aqualina Way',
  '8308 N  35th St',
  '2604 Notre Dame Ave',
  'The Delilah Plan in La Floresta Subdivision by Villa Homes',
  'Monte Plan in Campo de Suenos by Esperanza Homes',
  '3010 S  L Ln',
  '3006 S  L Ln',
  '2616 Brazos Ct',
  '2505 Swallow Ave',
  'San Juan Plan in Aqualina at Tres Lagos by Esperanza Homes',
  '2013 Queens Ave',
  '2004 Plan in La Floresta Subdivision by Villa Homes',
  '4909 N  47th St',
  '802 N  5th St',
  '4000 Zanesville Ave',
  '401 N  Ware Rd',
  '3909 Thunderbird Ave',
  '3012 Guadalupe',
  '4305 Xenops Ave',
  '4212 Xenops Ave',
  '1605 Hawk Ave',
  '4825 Estancia Pkwy',
  '10100 N  12th St',
  '9307 N  59th',
  '3412 Heron Ave',
  '900 E  Redbud Ave',
  '2105 Ebony Ave',
  'Guadalupe Plan in Aqualina at Tres Lagos by Esperanza Homes',
  '3120 Zurich Ave',
  '4300 Avocet Ave',
  '111 Yuma Ave',
  '4504 Estancia Pkwy',


In [12]:
# Create dataframe
df = pd.DataFrame(output)
df

,Address,Bathrooms,Area,Year Built,Parking,Price
0,8131 N 55th Ln,4 Baths,"2,200 sqft",2024,2 Car Garage,"$425,000"
1,3416 Toucan Ave,2 Baths,"2,491 sqft",2015,2 Car Garage,"$369,900"
2,4021 Zanesville Ave,3 Baths,"2,000 sqft",2023,2 Car Garage,"$349,000"
3,6308 N 20th St,3 Baths,"1,845 sqft",2004,Garage,"$209,900"
4,905 W Sycamore Ave,3 Baths,"1,485 sqft",2024,Garage,"$252,000"
5,14620 Aqualina Way,3 Baths,"2,142 sqft",2023,2 Car Garage,"$450,000"
6,8308 N 35th St,2 Baths,"1,819 sqft",,Attached Garage,"$252,866"
7,2604 Notre Dame Ave,2 Baths,"2,070 sqft",2005,2 Car Garage,"$295,000"
8,The Delilah Plan in La Floresta Subdivision by...,2 Baths,"2,199 sqft",2024,2 Car Garage,"$449,000+"
9,Monte Plan in Campo de Suenos by Esperanza Homes,3 Baths,"3,009 sqft",2024,Garage,"$305,990+"


In [13]:
# Add location to dataframe
df['Location'] = 'Mcallen'
df

,Address,Bathrooms,Area,Year Built,Parking,Price,Location
0,8131 N 55th Ln,4 Baths,"2,200 sqft",2024,2 Car Garage,"$425,000",Mcallen
1,3416 Toucan Ave,2 Baths,"2,491 sqft",2015,2 Car Garage,"$369,900",Mcallen
2,4021 Zanesville Ave,3 Baths,"2,000 sqft",2023,2 Car Garage,"$349,000",Mcallen
3,6308 N 20th St,3 Baths,"1,845 sqft",2004,Garage,"$209,900",Mcallen
4,905 W Sycamore Ave,3 Baths,"1,485 sqft",2024,Garage,"$252,000",Mcallen
5,14620 Aqualina Way,3 Baths,"2,142 sqft",2023,2 Car Garage,"$450,000",Mcallen
6,8308 N 35th St,2 Baths,"1,819 sqft",,Attached Garage,"$252,866",Mcallen
7,2604 Notre Dame Ave,2 Baths,"2,070 sqft",2005,2 Car Garage,"$295,000",Mcallen
8,The Delilah Plan in La Floresta Subdivision by...,2 Baths,"2,199 sqft",2024,2 Car Garage,"$449,000+",Mcallen
9,Monte Plan in Campo de Suenos by Esperanza Homes,3 Baths,"3,009 sqft",2024,Garage,"$305,990+",Mcallen


In [14]:
df.to_excel('dataframe_mcallen.xlsx', index=False)